GPU 연결을 추천드립니다!

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F


transform = transforms.Compose([transforms.ToTensor()])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [13]:
class VGGNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = self.conv_block(3, 64)
        self.conv2 = self.conv_block(64, 128)
        self.conv3 = self.conv_block(128, 256)

        self.fc1 = nn.Linear(4*4*256, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2))
        )

    def forward(self, inputs):
        batch_size = inputs.size(0)
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)

        x = x.view(batch_size, -1)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x

vgg_ = VGGNet(num_classes=10)
vgg_

VGGNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=4096, out_features=1024, bias=True)
  (fc2): Line

In [14]:
from torch import optim

# 모델 학습 함수
def training(model, train_loader, epochs=5):
    # 모델, 손실 함수, 옵티마이저 정의
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    model.to('cuda') # 모델 GPU 할당

    # 모델 훈련
    for epoch in range(epochs):  # 5번 학습
        model.train()
        for data, target in train_loader:
            data = data.to('cuda') # 이미지 GPU 할당
            target = target.to('cuda') # 라벨 GPU 할당
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

    # 학습을 마친 후엔 모델을 다시 cpu로 빼서 리소스를 아끼도록 합시다.
    model.to('cpu')
    print("Done!")

# 모델 테스트 함수
def evaluate(model, test_loader):
    model.eval()
    model.to('cuda')
    correct = 0
    total = 0
    with torch.no_grad():  # 테스트 중에는 기울기 계산 필요 없음
        for data, target in test_loader:
            data = data.to('cuda')
            target = target.to('cuda')
            output = model(data)
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    model.to('cpu')
    return accuracy

In [15]:
training(vgg_, train_loader)

Epoch 1, Loss: 2.3034
Epoch 2, Loss: 2.3028
Epoch 3, Loss: 2.3031
Epoch 4, Loss: 2.3012
Epoch 5, Loss: 2.2886
Done!


In [18]:
print("테스트셋 정확도 :", evaluate(vgg_, test_loader), "%")

테스트셋 정확도 : 18.17 %


In [40]:
class ResNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.input_conv = nn.Conv2d(3, 64, kernel_size=7, padding=3)

        self.conv1 = self.conv_block(64, 64)
        self.conv2 = self.conv_block(64, 128)
        self.conv3 = self.conv_block(128, 256)

        self.convert1 = nn.Conv2d(64, 64, kernel_size=1, stride=2)
        self.convert2 = nn.Conv2d(64, 128, kernel_size=1, stride=2)
        self.convert3 = nn.Conv2d(128, 256, kernel_size=1, stride=2)

        self.fc1 = nn.Linear(4*4*256, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

    def forward(self, inputs):
        batch_size = inputs.size(0)
        x = self.input_conv(inputs)

        res = x
        x = self.conv1(x)
        res = self.convert1(res)
        res += x

        x = self.conv2(res)
        res = self.convert2(res)
        res += x

        x = self.conv3(res)
        res = self.convert3(res)
        res += x

        x = res.view(batch_size, -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x

resnet_ = ResNet(num_classes=10)
resnet_

ResNet(
  (input_conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv1): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
  )
  (conve

In [41]:
training(resnet_, train_loader)

Epoch 1, Loss: 1.0201
Epoch 2, Loss: 1.1092
Epoch 3, Loss: 1.0091
Epoch 4, Loss: 0.8364
Epoch 5, Loss: 1.1521
Done!


In [42]:
print("테스트셋 정확도 :", evaluate(resnet_, test_loader), "%")

테스트셋 정확도 : 55.92 %
